In [1]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
import numpy as np
from matplotlib import pyplot as plt
import aocutils
import networkx as nx
import itertools

In [2]:
from collections import namedtuple
from aocutils.maze import bfs
from aocutils.common import multimap
from aocutils.math import all_combinations

In [3]:
moves = [(i, i+delta) for i in range(4) for delta in [-1,1] if 0 <= i + delta < 4]
moves = multimap(moves)


from itertools import permutations, compress
from functools import cache
def splititer(it, amount):
    # split iterable all unique permutations
    @cache
    def rev(mask):
        return (not idx for idx in mask)
    n = len(it)
    
    options= {mask for mask in itertools.permutations([1]*amount +[0]*(n-amount))}
    return [(tuple(compress(it, o)), tuple(compress(it, rev(o)))) for o in options]
splititer('abcde',2)


[(('a', 'c'), ('b', 'd', 'e')),
 (('a', 'b'), ('c', 'd', 'e')),
 (('c', 'e'), ('a', 'b', 'd')),
 (('b', 'e'), ('a', 'c', 'd')),
 (('a', 'e'), ('b', 'c', 'd')),
 (('c', 'd'), ('a', 'b', 'e')),
 (('d', 'e'), ('a', 'b', 'c')),
 (('b', 'd'), ('a', 'c', 'e')),
 (('b', 'c'), ('a', 'd', 'e')),
 (('a', 'd'), ('b', 'c', 'e'))]

In [4]:
splititer('abc',2)

[(('a', 'c'), ('b',)), (('a', 'b'), ('c',)), (('b', 'c'), ('a',))]

In [39]:
from copy import deepcopy
def load(floor): # returns loadedfloor and passengers
    amount_gen = floor.pair + len(floor.gen)
    
    NEWVARS = 'XY'
    for i in range(min(floor.pair, 2)):
        floor.pair -= 1
        floor.chip += NEWVARS[i]
        floor.gen += NEWVARS[i]
        
    newfloors = []
    if floor.chip:                
        # take all single chips with you
        for amount in range(1, min(len(floor.chip)+1, 3)):
            for passengers, newchip in splititer(floor.chip,amount):
                n = deepcopy(floor)
                n.chip = ''.join(newchip)
                for i in range(2):
                    if NEWVARS[i] in n.chip and NEWVARS[i] in n.gen:
                        
                        n.chip = n.chip.replace(NEWVARS[i], '')
                        n.gen = n.gen.replace(NEWVARS[i], '')
                        n.pair += 1   
                    
                newfloors.append((n, passengers))
    if floor.gen and 1==2:
        # take all single gens with you
        # take all combinations of 2 gens with you
        for amount in range(1, min(len(floor.gen)+1, 3)):
            for passengers, newchip in splititer(floor.gen,amount):
                n = deepcopy(floor)
                n.gen = ''.join(newchip)
                for i in range(2):
                    if NEWVARS[i] in n.chip and NEWVARS[i] in n.gen:
                        n.chip = n.chip.replace(NEWVARS[i], '')
                        n.gen = n.gen.replace(NEWVARS[i], '')
                        n.pair += 1    
                newfloors.append((n, passengers))
    if floor.pair:
        pass    
        # take a pair with you
        # take one microchip
        # take two microchips
        # take a generator if there is only 1 pair and 0 generators
        # take two generators
    return newfloors
    
load(Floor(2, 'BC', 'A'))
    



[(Floor(pair=1, chip='BC', gen='AY'), ('Y',)),
 (Floor(pair=2, chip='B', gen='A'), ('C',)),
 (Floor(pair=2, chip='C', gen='A'), ('B',)),
 (Floor(pair=1, chip='BC', gen='AX'), ('X',)),
 (Floor(pair=1, chip='B', gen='AY'), ('C', 'Y')),
 (Floor(pair=2, chip='', gen='A'), ('B', 'C')),
 (Floor(pair=1, chip='B', gen='AX'), ('C', 'X')),
 (Floor(pair=1, chip='C', gen='AX'), ('B', 'X')),
 (Floor(pair=1, chip='C', gen='AY'), ('B', 'Y')),
 (Floor(pair=0, chip='BC', gen='AXY'), ('X', 'Y'))]

In [23]:
@dataclass
class State:
    floors: list
    elev: int
    
@dataclass
class Floor:
    pair: int
    chip: str
    gen: str
    

In [9]:
def unload(floor, cargo):
    pass
    
s = State((Floor(2,0,0), Floor(2,'A',0), Floor(0,0,'A'), Floor(0,0,0)), 0)
floors = s.floors
elev = s.elev
neigh = []
for loadedfloor, passengers in load(floors[elev]): # for every combination of passengers that can be loaded
    for flooridx in moves[elev]: # for every floor to move to
        if unloadedfloor := unload(floors[flooridx], passengers): # if offloading is valid
            neigh.append(constructfloors(floors, loadedfloor, elev, unloadedfloor, flooridx))
neigh

TypeError: object of type 'int' has no len()

In [65]:
from itertools import combinations
import heapq

# hardcoded input
polonium, thulium, promethium, ruthenium, cobalt, elerium, dilithium = 1, 2, 3, 4, 5, 6, 7
initial = (0, (
	tuple(sorted((polonium, thulium, -thulium, promethium, ruthenium, -ruthenium, cobalt, -cobalt, elerium, -elerium, dilithium, -dilithium))),
	tuple(sorted((-polonium, -promethium))), (), ()
))

initial = (0, (
	tuple(sorted((1,2,-1,-2))),
	tuple(sorted((-3,3,4,-4,-5))), (5), ()
))


print(initial)
def correct(floor):
	if not floor or floor[-1] < 0: # no generators
		return True
	return all(-chip in floor for chip in floor if chip < 0)

frontier = []
heapq.heappush(frontier, (0, initial))
cost_so_far = {initial: 0}

while frontier:
	print(len(frontier))
	_, current = heapq.heappop(frontier)
	floor, floors = current
	if floor == 3 and all(len(f) == 0 for f in floors[:-1]): # goal!
		print('goal')
		break

	directions = [dir for dir in (-1, 1) if 0 <= floor + dir < 4]
	moves = list(combinations(floors[floor], 2)) + list(combinations(floors[floor], 1))
	for move in moves:
		for direction in directions:
			print('hi')
			new_floors = list(floors)
			new_floors[floor] = tuple(x for x in floors[floor] if x not in move)
			new_floors[floor+direction] = tuple(sorted(floors[floor+direction] + move))

			if not correct(new_floors[floor]) or not correct(new_floors[floor+direction]):
				continue
			print('hi')
			next = (floor+direction, tuple(new_floors))
			new_cost = cost_so_far[current] + 1
			if next not in cost_so_far or new_cost < cost_so_far[next]:
				cost_so_far[next] = new_cost
				priority = new_cost - len(new_floors[3])*10 # silly manually tweakable heuristic factor
				heapq.heappush(frontier, (priority, next))

print(cost_so_far[current], current)

(0, ((-2, -1, 1, 2), (-5, -4, -3, 3, 4), 5, ()))
1
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
0 (0, ((-2, -1, 1, 2), (-5, -4, -3, 3, 4), 5, ()))


In [52]:
list(combinations((-1,-2,-3,-4,-5), 2))

[(-1, -2),
 (-1, -3),
 (-1, -4),
 (-1, -5),
 (-2, -3),
 (-2, -4),
 (-2, -5),
 (-3, -4),
 (-3, -5),
 (-4, -5)]

In [4]:
0 ^8 ^10

2

In [8]:
import calendar


In [11]:
from datetime import datetime

In [18]:

(datetime(2022,8,2) - datetime(2022,8,1))

TypeError: 'datetime.timedelta' object is not iterable

In [ ]:
0   1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26  27 28 29 30 31 32